In [18]:
import os
access_token = os.environ["ACCESS_TOKEN"]
account_id_demo = os.environ["ACCOUNT_DEMO"]
account_id_hedge = os.environ["ACCOUNT_HEDGE"]
import oandapyV20
import oandapyV20.endpoints.transactions as trans
import pandas as pd

In [325]:
client = oandapyV20.API(access_token=access_token)
param = { "id": 900 }
r = trans.TransactionsSinceID(accountID=account_id_demo,  params=param)
trades= client.request(r)
param1 = { "id": 0 }
request_hedge = trans.TransactionsSinceID(accountID=account_id_hedge,  params=param1)
trades_hedge= client.request(request_hedge)

In [326]:
all_transactions_hedge = pd.DataFrame(trades_hedge['transactions'])
all_transactions = pd.DataFrame(trades['transactions'])
all_transactions = pd.concat([all_transactions_hedge,all_transactions])
all_transactions['time'] = pd.to_datetime(all_transactions['time'])
all_transactions['date'] = all_transactions['time'].dt.date
all_transactions = all_transactions[all_transactions['date'] == all_transactions['date'].max()]
all_transactions['pl'] = pd.to_numeric(all_transactions['pl'])
print(all_transactions[all_transactions['reason'].isin(['TRAILING_STOP_LOSS_ORDER', 'TAKE_PROFIT_ORDER'])].groupby(['accountID','reason'])['pl'].sum())
print(all_transactions['pl'].sum())

accountID             reason                  
101-004-31059296-001  TAKE_PROFIT_ORDER           2353.7937
                      TRAILING_STOP_LOSS_ORDER   -5911.3750
101-004-31059296-003  TAKE_PROFIT_ORDER           3757.7192
                      TRAILING_STOP_LOSS_ORDER   -4347.5184
Name: pl, dtype: float64
-2733.718300000001


In [80]:
pd.to_numeric(all_transactions['halfSpreadCost']).sum()

4631.0979

In [328]:
orders = pd.read_csv('/work/data/orders/all_orders.csv')
orders.rename(columns={'Type':'TypeB'},inplace=True)
orders['Time'] = pd.to_datetime(orders['Time'])
all_transactions['id'] = pd.to_numeric(all_transactions['id'])
all_transactions['batchID'] = pd.to_numeric(all_transactions['batchID'])
all_transactions = pd.merge(all_transactions,orders,left_on='batchID', right_on = 'Order_ID',how='left')

In [332]:
trans_orders = _dntk.execute_sql(
  'WITH trading_orders AS (\n    SELECT \n        CAST(all1.batchID AS INTEGER) as batchID,base.accountID,all1.Strategy,all1.TypeB,\n        base.pl,\n        date_trunc(\'minute\',all1.time) time_tf,\n        date_trunc(\'minute\', all1.time - INTERVAL \'6 minutes\') AS order_time\n    FROM all_transactions base \n    LEFT JOIN all_transactions order_filled \n        ON CAST(base.orderID AS INTEGER) = order_filled.id\n    LEFT JOIN all_transactions all1 \n        ON order_filled.batchID = all1.id\n    WHERE base.reason IN (\'TAKE_PROFIT_ORDER\', \'TRAILING_STOP_LOSS_ORDER\')\n)\nSELECT * FROM trading_orders\nORDER BY batchID ',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
trans_orders

,batchID,accountID,Strategy,TypeB,pl,time_tf,order_time
0,64,101-004-31059296-003,volume_based_hedge,BUY,889.4796,2025-04-08 10:21:00+00:00,2025-04-08 10:15:00+00:00
1,70,101-004-31059296-003,volume_based_hedge,BUY,-425.3835,2025-04-08 10:36:00+00:00,2025-04-08 10:30:00+00:00
2,76,101-004-31059296-003,volume_based_hedge,BUY,-456.8629,2025-04-08 10:46:00+00:00,2025-04-08 10:40:00+00:00
3,83,101-004-31059296-003,volume_based_hedge,BUY,31.1602,2025-04-08 11:16:00+00:00,2025-04-08 11:10:00+00:00
4,87,101-004-31059296-003,volume_based_hedge,BUY,-298.9971,2025-04-08 11:21:00+00:00,2025-04-08 11:15:00+00:00
...,...,...,...,...,...,...,...
56,1214,101-004-31059296-001,volume_based,BUY,-361.7824,2025-04-08 16:51:00+00:00,2025-04-08 16:45:00+00:00
57,1220,101-004-31059296-001,volume_based,BUY,233.6494,2025-04-08 16:56:00+00:00,2025-04-08 16:50:00+00:00
58,1228,101-004-31059296-001,volume_based,SELL,264.7012,2025-04-08 17:36:00+00:00,2025-04-08 17:30:00+00:00
59,1232,101-004-31059296-001,volume_based,SELL,-338.1336,2025-04-08 17:41:00+00:00,2025-04-08 17:35:00+00:00


In [338]:
strategy_agg=pd.merge(    
    trans_orders[trans_orders['accountID']=='101-004-31059296-003'],
    trans_orders[trans_orders['accountID']=='101-004-31059296-001'],
    on = 'time_tf', how='left')

In [359]:
prediction_actual.head()

,datetime,upward_move_6,downward_move_6,upward_move_20,downward_move_20,bearish_move_flag_20,bearish_move_flag_20.1,y_actual
0,2025-04-03 06:20:00+00:00,51.8,0.6,53.1,0.6,0,0,1
1,2025-04-03 06:25:00+00:00,41.5,6.9,41.6,6.9,0,0,1
2,2025-04-03 06:30:00+00:00,25.9,4.1,25.9,21.6,2,2,3
3,2025-04-03 06:35:00+00:00,12.3,7.2,12.3,35.2,2,2,2
4,2025-04-03 06:40:00+00:00,12.6,8.3,16.3,34.9,2,2,2


In [348]:
strategy_agg['pl_x'] = pd.to_numeric(strategy_agg['pl_x'])
strategy_agg['pl_y'] = pd.to_numeric(strategy_agg['pl_y'])
strategy_agg['pl_agg'] = strategy_agg['pl_x'] + strategy_agg['pl_y']
strategy_agg = strategy_agg[(strategy_agg['time_tf'].dt.date == strategy_agg['time_tf'].dt.date.max())
            ][['Strategy_x','pl_x','pl_y','pl_agg','order_time_x','TypeB_x']]

In [365]:
prediction_actual = pd.read_csv('/work/data/predictions/predictions_actuals.csv')
strategy_agg['order_time_x'] = pd.to_datetime(strategy_agg['order_time_x'])
prediction_actual['datetime'] = pd.to_datetime(prediction_actual['datetime'])
pd.merge(strategy_agg,prediction_actual,left_on='order_time_x',right_on = 'datetime')

,Strategy_x,pl_x,pl_y,pl_agg,order_time_x,TypeB_x,datetime,upward_move_6,downward_move_6,upward_move_20,downward_move_20,bearish_move_flag_20,bearish_move_flag_20.1,y_actual
0,volume_based_hedge,889.4796,NaN,NaN,2025-04-08 10:15:00+00:00,BUY,2025-04-08 10:15:00+00:00,22.1,0.1,27.7,0.1,0,0,1
1,volume_based_hedge,-425.3835,NaN,NaN,2025-04-08 10:30:00+00:00,BUY,2025-04-08 10:30:00+00:00,6.6,10.4,22.6,10.4,0,0,1
2,volume_based_hedge,-456.8629,NaN,NaN,2025-04-08 10:40:00+00:00,BUY,2025-04-08 10:40:00+00:00,6.0,11.2,21.8,11.2,0,0,1
3,volume_based_hedge,31.1602,-519.1095,-487.9493,2025-04-08 11:10:00+00:00,BUY,2025-04-08 11:10:00+00:00,6.2,13.6,30.0,13.6,0,0,1
4,volume_based_hedge,-298.9971,1169.0982,870.1011,2025-04-08 11:15:00+00:00,BUY,2025-04-08 11:15:00+00:00,2.2,17.6,26.0,17.6,2,2,3
5,volume_based_hedge,-472.2097,1184.6955,712.4858,2025-04-08 11:20:00+00:00,BUY,2025-04-08 11:20:00+00:00,0.4,17.0,26.6,17.0,2,2,3
6,volume_based_hedge,-377.9202,77.9152,-300.0050,2025-04-08 11:25:00+00:00,BUY,2025-04-08 11:25:00+00:00,6.8,10.6,33.0,10.6,0,0,1
7,volume_based_hedge,15.5799,NaN,NaN,2025-04-08 11:35:00+00:00,BUY,2025-04-08 11:35:00+00:00,23.7,2.2,41.4,2.2,0,0,1
8,volume_based_hedge,-361.6876,871.8033,510.1157,2025-04-08 11:50:00+00:00,SELL,2025-04-08 11:50:00+00:00,17.4,4.1,30.8,4.1,0,0,1
9,volume_based_hedge,-251.7712,186.8545,-64.9167,2025-04-08 12:45:00+00:00,BUY,2025-04-08 12:45:00+00:00,0.6,16.0,0.6,53.8,2,2,2


In [354]:
orders[(orders['Time'].dt.date == orders['Time'].dt.date.max()) 
        & (orders['Account_ID'] == '101-004-31059296-003')][['Order_ID','datetime']].head()

KeyError: "['datetime'] not in index"

In [256]:
strategy_agg[(strategy_agg['time_tf'].dt.date == strategy_agg['time_tf'].dt.date.max())
            ][['pl_agg']].mean()

pl_agg   -56.463657
dtype: float64

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a127aa9e-0a77-4af9-a6ce-85e7a9b74042' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>